<a href="https://colab.research.google.com/github/LLT9798/Twitter-Demography-Research-Project/blob/master/Test1_Contd_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import re
import nltk
import seaborn as sns
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn import metrics
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC


import tensorflow.compat.v1 as tf
print(tf.__version__)
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras import optimizers





from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

1.15.0


In [3]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Get data from Google drive
link = 'https://drive.google.com/open?id=1A1sv29Np75mWPhz9nVk35UptWDztNHp3'

fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('791531.csv')  

df = pd.read_csv('791531.csv', encoding='latin-1')
df.shape

1A1sv29Np75mWPhz9nVk35UptWDztNHp3


(10020, 3)

In [0]:
df =df.drop(columns='Unnamed: 0')

In [5]:
df.head(10)

,text,gender
0,Robbie E Responds To Critics After Win Against...,1
1,ÂÃÃIt felt like they were my friends and I ...,1
2,Hi @JordanSpieth - Looking at the url - do you...,1
3,Watching Neighbours on Sky+ catching up with t...,0
4,"Ive seen people on the train with lamps, chair...",0
5,Gala Bingo clubs bought for Ã¥Â£241m: The UK's...,1
6,@_Aphmau_ the pic defines all mcd fangirls/fan...,0
7,@Evielady just how lovely is the tree this yea...,0
8,https://t.co/EROGWTFTYo It's a glow of satisfa...,0
9,@giannaaa28 lmao _ÃÃ·Ã¢_ÃÃ·Ã¢ dude I'm hella...,0


In [0]:
def to_wordlist( review, remove_stopwords=True):
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()

    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)

    # 3. Convert words to lower case and split them
    words = review_text.lower().split()

    # 4. Optionally remove stop words (Apply NLTK)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer #PorterStemmer()
    for word in words:
        b.append(stemmer.stem(word))

    # 5. Return a list of words
    return(b)

In [0]:
train, test = train_test_split(df, test_size = 0.2)

In [0]:
clean_train_text = []
for text in train['text']:
    clean_train_text.append( " ".join(to_wordlist(text)))
    
clean_test_text = []
for text in test['text']:
    clean_test_text.append( " ".join(to_wordlist(text)))

In [0]:
vectorizer = CountVectorizer( min_df=2, max_df=0.95, max_features = 200000, ngram_range = ( 1, 4 ))

vectorizer = vectorizer.fit(clean_train_text)
train_features = vectorizer.transform(clean_train_text)

test_features = vectorizer.transform(clean_test_text)

In [0]:
fselect = SelectKBest(chi2 , k=10000)
train_features = fselect.fit_transform(train_features, train['gender'])
test_features = fselect.transform(test_features)

In [0]:
model1 = MultinomialNB(alpha=0.001)
model1.fit( train_features, train['gender'] )

model2 = SGDClassifier(loss='modified_huber', random_state=100, shuffle=True)
model2.fit( train_features, train['gender'] )

model3 = RandomForestClassifier()
model3.fit( train_features, train['gender'] )

model4 = GradientBoostingClassifier()
model4.fit( train_features, train['gender'] )

pred_1 = model1.predict( test_features.toarray() )
pred_2 = model2.predict( test_features.toarray() )
pred_3 = model3.predict( test_features.toarray() )
pred_4 = model4.predict( test_features.toarray() )

In [58]:
print('prediction 1 accuracy: ', accuracy_score(test['gender'], pred_1))
print('prediction 2 accuracy: ', accuracy_score(test['gender'], pred_2))
print('prediction 3 accuracy: ', accuracy_score(test['gender'], pred_3))
print('prediction 4 accuracy: ', accuracy_score(test['gender'], pred_4))

prediction 1 accuracy:  0.5863273453093812
prediction 2 accuracy:  0.5788423153692615
prediction 3 accuracy:  0.5873253493013972
prediction 4 accuracy:  0.5643712574850299


In [0]:
batch_size = 100
nb_classes = 2

In [62]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (8016, 1000)
X_test shape: (2004, 1000)


In [83]:
train_features[:].data

array([0.21266613, 0.2129963 , 0.2129963 , ..., 0.53698325, 0.65345834,
       0.53351775])

In [87]:
y_train = np.array(train['gender'])
y_test = np.array(test['gender'])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.mean(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(256, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
optimizer = optimizers.SGD(lr=0.25, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, Y_train, nb_epoch= 30, batch_size=30, validation_split=0.1)

preds = model.predict_classes(X_test, verbose=0)
print(preds)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7214 samples, validate on 802 samples
Epoch 1/30
7214/7214 [==============================] - 4s 488us/step - loss: 0.7031 - acc: 0.5168 - val_loss: 0.7212 - val_acc: 0.5224
Epoch 2/30
7214/7214 [==============================] - 2s 217us/step - loss: 0.6991 - acc: 0.5222 - val_loss: 0.7045 - val_acc: 0.4776
Epoch 3/30
7214/7214 [==============================] - 2s 220us/step - loss: 0.6962 - acc: 0.5130 - val_loss: 0.6987 - val_acc: 0.5224
Epoch 4/30
7214/7214 [==============================] - 2s 217us/step - loss: 0.6963 - acc: 0.5229 - val_loss: 0.6947 - val_acc: 0.5224
Epoch 5/30
7214/7214 [==============================] - 2s 221us/step - loss: 0.6988 - acc: 0.5198 - val_loss: 0.7069 - val_acc: 0.5224
Epoch 6/30
7214/7214 [==============================] - 2s 220us/step - loss: 0.6975 - acc: 0.5200 - val_loss: 0.6929 - val_acc: 0.5224
Epoch 7/30
7214/7214 [==============================] - 2s 228us/step - loss: 0.6945 - acc: 0.5291 - val_loss: 0.6979 - val_acc: 0.5224
E

In [88]:
print('prediction 5 accuracy: ', accuracy_score(test['gender'], preds))

prediction 5 accuracy:  0.5214570858283433


In [89]:
metrics.f1_score(test['gender'], preds, average='weighted')

0.35744352554320674